# Keras Neural Network

## Initialisations

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd

In [3]:
# Critere de performance
def compute_pred_score(y_true, y_pred):
    y_pred_unq =  np.unique(y_pred)
    for i in y_pred_unq:
        if((i != -1) & (i!= 1) & (i!= 0) ):
            raise ValueError('The predictions can contain only -1, 1, or 0!')
    y_comp = y_true * y_pred
    score = float(10*np.sum(y_comp == -1) + np.sum(y_comp == 0))
    score /= y_comp.shape[0]
    return score

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

## Data Loading

In [4]:
# loaddataset
X_train_fname = 'training_templates.csv'
y_train_fname = 'training_labels.txt'
X_test_fname  = 'testing_templates.csv'
X_train = pd.read_csv(X_train_fname, sep=',', header=None).values
X_test  = pd.read_csv(X_test_fname,  sep=',', header=None).values
y_train = np.loadtxt(y_train_fname, dtype=np.int)

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)

pca = PCA()  
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print X_train.shape

(105600, 128)


## GridSearch: Epochs & Batch_size

In [5]:
from keras.optimizers import SGD
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, init='glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Dense(128, input_dim=128, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    #optimizer = SGD(lr=learn_rate)
    model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# grid search epochs, batch size and optimizer
epochs = [14,18]
batches = [250,350]
param_grid = dict(epochs=epochs, batch_size=batches)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print("Done fitting")

Done fitting


In [6]:
print grid_result.best_params_

{'epochs': 18, 'batch_size': 250}


In [7]:
model0 = KerasClassifier(build_fn=create_model, batch_size=250, epochs=18, verbose=0)
model0.fit(X_train, y_train)

In [8]:
y_pred = model0.predict(X_test)
y_predict_proba = model0.predict_proba(X_test)[:,0]

for i in range(len(y_pred)):
    if (y_predict_proba[i]<0.9)and(y_predict_proba[i]>1-0.9):
        y_pred[i]=0
print np.unique(y_pred)

np.savetxt('y_pred_e18_bs250.txt', y_pred, fmt='%d')
# batch_size=300, epochs=10 seuil = 0.9
# batch_size=350, epochs=14 seuil = 0.97

[-1  0  1]


## GridSearch: Dropout rate

In [9]:
from keras.layers import Dropout
from keras.constraints import maxnorm

np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model2(dropout_rate=0.0, weight_constraint=0):
    # create model
    model = Sequential()
    model.add(Dense(128, input_dim=128, kernel_initializer="uniform", activation='relu',kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer="uniform", activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer="SGD", metrics=['accuracy'])
    return model

model2 = KerasClassifier(build_fn=create_model2, batch_size=250, epochs=18, verbose=1)

weight_constraint = [2, 3]
dropout_rate = [0.2, 0.3]
param_grid2 = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid2 = GridSearchCV(estimator=model2, param_grid=param_grid2)
grid_result2 = grid2.fit(X_train, y_train)

Epoch 1/18
70400/70400 [==============================] - 2s - loss: 0.5636 - acc: 0.7459       

In [ ]:
print grid_result2.best_params_

In [ ]:
# predictions
model2 = KerasClassifier(build_fn=create_model, batch_size=250, epochs=8, verbose=0)
model2.fit(X_train, y_train)
y_pred_train =  model2.predict(X_train)
y_predict_train_proba = model2.predict_proba(X_train)[:,0]
        
for i in range(len(y_pred_train)):
    if (y_predict_train_proba[i]<0.9)and(y_predict_train_proba[i]>1-0.9):
        y_pred_train[i]=0

# score
score = compute_pred_score(y_train, y_pred_train)
print('Score sur le train : %s' % score)